### Workflow to get the trial and fixation tables
1. Run ProcessMonkeyGameSingleSubject in Matlab on the TrialSubject Folder from the Experiment
2. Run the Replay app on the file in the ProcessedData Folder
3. Run the Replay script in Matlab on the Replayer generated file
4. Save FixationDetailsForAnalysis and TrialData as structs
5. Load the two .mat files into Python using the custom loadmat script

### Notes/References

http://stackoverflow.com/questions/15512560/access-mat-file-containing-matlab-classes-in-python
http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/

### Loading Libraries

In [1]:
import scipy.io as sio
import pandas as pd
import tables
import numpy as np
import os

### Custom Functions for loading matlab structs

In [2]:
# http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], sio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, sio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

### Setting Jupyter Notebook settings

In [173]:
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 50)

### Data Preprocessing
Mostly going through the matlab files

In [174]:
foldername = './ProcessedStructsForPython/'
trials_list = []
fixations_list = []
matrix_list = []
for _file in os.listdir(foldername):
    if _file.find('Trials') + 1:
        t = loadmat(foldername+_file)
        mdata = t['t']
        l = []
        for i in mdata['data']:
            s = pd.Series(i)
            l.append(s.to_frame())
        trials_df = pd.concat(l, axis=1)
        trials_df.columns = mdata['varnames']
        trials_list.append(trials_df)
    elif _file.find('Fixations') + 1:
        f = loadmat(foldername+_file)
        mdata = f['f']
        l = []
        for i in mdata['data']:
            s = pd.Series(i)
            l.append(s.to_frame())
        fixations_df = pd.concat(l, axis=1)
        fixations_df.columns = mdata['varnames']
        fixations_df.drop(fixations_df.columns[[14, 15, 16]], axis=1, inplace=True)
        fixations_df = fixations_df[pd.notnull(fixations_df['TrialInExperiment'])]
        fixations_df = fixations_df[pd.notnull(fixations_df['Accuracy'])]
        fixations_df['Block'] = fixations_df['Block'].astype(int)
        fixations_df['TrialInBlock'] = fixations_df['TrialInBlock'].astype(int)
        fixations_df['TrialInExperiment'] = fixations_df['TrialInExperiment'].astype(int)
        fixations_df['Accuracy'] = fixations_df['Accuracy'].astype(int)
        fixations_df['FixCategory'] = fixations_df['FixCategory'].astype(int)
        fixations_df['PrePickup'] = fixations_df['PrePickup'].astype(int)
        fixations_df['Epoch'] = fixations_df['Epoch'].astype(int)
        fixations_df = fixations_df.loc[fixations_df['PrePickup'] == 1] # http://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
        fixations_df = fixations_df.loc[fixations_df['Epoch'] == 2] # 2 is from fixation door opens to reward door
        fixations_list.append(fixations_df)
        
        df = fixations_df
        
        fixCat = df['Duration'].groupby([df['Block'], df['TrialInBlock'], df['FixCategory']]).count().unstack()
        fixCat.reset_index(inplace=True)
        
        columns = []
        for i in fixCat.columns:
            if type(i) == int:
                if len(str(i)) == 1:
                    columns.append('FixCat ' + '0' + str(i) + ' Count')
                else:
                    columns.append('FixCat ' + str(i) + ' Count')
            elif i == 'TrialInBlock':
                columns.append("Block's Trial")
            else:
                columns.append(i)
        fixCat.columns = columns
        
        fixCat_dur = df['Duration'].groupby([df['Block'], df['TrialInBlock'], df['FixCategory']]).mean().unstack()
        fixCat_dur.reset_index(inplace=True)
        
        columns = []
        for i in fixCat_dur.columns:
            if type(i) == int:
                if len(str(i)) == 1:
                    columns.append('FixCat ' + '0' + str(i) + ' Duration')
                else:
                    columns.append('FixCat ' + str(i) + ' Duration')
            elif i == 'TrialInBlock':
                columns.append("Block's Trial")
            else:
                columns.append(i)
        fixCat_dur.columns = columns
        
        matrix = fixCat_dur.add(fixCat[list(np.arange(2,  len(fixCat.columns)))], axis=1, fill_value=0)
        matrix_list.append(matrix)
    else:
        pass
fm_list = []
for i in range(0, len(matrix_list)):
    matrix_list[i]['Acc Smoothed'] = trials_list[i]['SmoothAcc']
    #matrix_list[i]['Beta 0'] = [1] * len(matrix_list[i])
    matrix = matrix_list[i]
    matrix = matrix.reindex_axis(sorted(matrix.columns), axis=1)
    matrix = matrix[list(matrix.columns[:7])]
    matrix = matrix.dropna()
    fm_list.append(matrix)

In [175]:
# Print how many subjects
print len(fm_list)

5


### The Final Feature Matrix

In [176]:
# Features to add: Exploration Time
matrix = pd.concat(fm_list, axis=0, ignore_index=False)
matrix

,Acc Smoothed,Block,Block's Trial,FixCat 01 Count,FixCat 01 Duration,FixCat 02 Count,FixCat 02 Duration
1,0.500000,1.0,2.0,2.0,0.423266,3.0,0.298814
2,0.500000,1.0,4.0,13.0,0.376860,4.0,0.346617
3,0.666667,1.0,5.0,12.0,0.426565,11.0,0.387171
4,0.500000,1.0,6.0,26.0,0.249824,16.0,0.271624
5,0.750000,1.0,7.0,4.0,0.229060,5.0,0.232654
6,0.500000,1.0,8.0,6.0,0.346647,3.0,0.508816
7,0.600000,1.0,9.0,6.0,0.294403,3.0,0.398833
8,0.600000,1.0,10.0,3.0,0.254448,1.0,0.339944
9,0.800000,1.0,11.0,5.0,0.405277,4.0,0.219104
10,0.800000,1.0,12.0,3.0,0.380946,1.0,0.290063


## Machine Learning

In [177]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score

### All Subjects or

In [178]:
X = matrix[matrix.columns[1:]]
y = matrix[matrix.columns[0:1]]

### Individual Subject
Please specify k

In [107]:
k = 1
X = fm_list[k][fm_list[k].columns[1:]]
y = fm_list[k][fm_list[k].columns[0:1]]

### Create training and test sets

In [152]:
# https://stats.stackexchange.com/questions/249892/wildly-different-r2-between-statsmodels-linear-regression-and-sklearn-linear/249897
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Scikit Learn's Linear Regression (No Intercept)

In [166]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [167]:
y_predict = lr.predict(X_test)

In [168]:
print "Linear Regression Score: %f" % (lr.score(X_test, y_test))
print "R^2 Score: %f" % (r2_score(y_test, y_predict))
print "Linear Regression Coefficients: %s" % (lr.coef_)

Linear Regression Score: -0.234067
R^2 Score: -0.234067
Linear Regression Coefficients: [[ 0.03091071  0.00309338  0.02831667  0.66921835 -0.01776865  0.51699073]]


### StatsModel's Linear Regression (No Intercept)

In [169]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Acc Smoothed   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     179.6
Date:                Fri, 14 Apr 2017   Prob (F-statistic):           1.22e-69
Time:                        23:57:34   Log-Likelihood:                -19.450
No. Observations:                 171   AIC:                             50.90
Df Residuals:                     165   BIC:                             69.75
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Block                  0.0180      0

### StatsModel's Linear Regression (With Intercept)

In [170]:
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Acc Smoothed   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     5.743
Date:                Fri, 14 Apr 2017   Prob (F-statistic):           1.91e-05
Time:                        23:57:46   Log-Likelihood:                -4.0331
No. Observations:                 171   AIC:                             22.07
Df Residuals:                     164   BIC:                             44.06
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
const                  0.6000      0

In [171]:
print('Parameters: ', results.params)
print('R2: ', results.rsquared)

('Parameters: ', const                 0.599976
Block                 0.005140
Block's Trial         0.004231
FixCat 01 Count       0.033347
FixCat 01 Duration    0.122524
FixCat 02 Count      -0.034385
FixCat 02 Duration   -0.258824
dtype: float64)
('R2: ', 0.17361949183791892)


In [139]:
# https://stats.stackexchange.com/questions/146804/difference-between-statsmodel-ols-and-scikit-linear-regression
predictions = results.predict(X_test)

In [159]:
#prediction_error = y_test - predictions

### SVM or SVR for Regression

In [14]:
from sklearn.svm import SVR
svr = SVR(kernel='linear')

In [15]:
svr.fit(X_train, y_train)

/Users/chensteven/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [16]:
svr.score(X_test, y_test)

0.035496121099233058

In [17]:
svr.coef_

array([[ 0.01003707,  0.00034932,  0.01673799, -0.03490702, -0.0299696 ,
        -0.13513888]])

## Cross Validation